In [ ]:
import pickle
import numpy as np
import pandas as pd
import json

# Load symptom index
with open("symptom_index.json") as f:
    symptoms = json.load(f)

# Load model
model = pickle.load(open('ExtraTrees', 'rb'))

# Daftar penyakit sesuai index di model
diseases = [
    '(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne', 'Alcoholic hepatitis', 'Allergy', 
    'Arthritis', 'Bronchial Asthma', 'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis', 
    'Common Cold', 'Dengue', 'Diabetes', 'Dimorphic hemmorhoids(piles)', 'Drug Reaction', 
    'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack', 'Hepatitis B', 'Hepatitis C', 
    'Hepatitis D', 'Hepatitis E', 'Hypertension', 'Hyperthyroidism', 'Hypoglycemia', 'Hypothyroidism', 
    'Impetigo', 'Jaundice', 'Malaria', 'Migraine', 'Osteoarthristis', 'Paralysis (brain hemorrhage)', 
    'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis', 'Typhoid', 
    'Urinary tract infection', 'Varicose veins', 'hepatitis A'
]

# Ini list symptoms dari lo
test_symptoms = ["itching", "family_history", "mucoid_sputum", 
"rusty_sputum", "lack_of_concentration", "visual_disturbances", "receiving_blood_transfusion", 
"receiving_unsterile_injections", "coma", "stomach_bleeding", "distention_of_abdomen", 
"history_of_alcohol_consumption", "blood_in_sputum", "prominent_veins_on_calf", "palpitations", 
"painful_walking", "pus_filled_pimples", "blackheads"]

# Encode symptoms
features = pd.Series([0]*len(symptoms), index=symptoms)
for symptom in test_symptoms:
    if symptom in features.index:
        features[symptom] = 1
    else:
        print(f"❗ Symptom not found in index: {symptom}")

# Reshape
features = features.to_numpy().reshape(1, -1)

# Predict
proba = model.predict_proba(features)
top5_idx = np.argsort(proba[0])[-5:][::-1]
top5_proba = np.sort(proba[0])[-5:][::-1]

print("\n🎯 Top 5 Predictions:")
for i in range(5):
    print(f"{diseases[top5_idx[i]]} → {top5_proba[i]:.4f}")


In [2]:
import requests
import json

# Ganti sesuai dengan list symptoms valid (harus snake_case, sesuai model training)
test_symptoms = [
    "dehydration", "indigestion", "headache", "nausea", "vomiting",
]

# Kirim POST request ke Flask API
url = 'http://127.0.0.1:5050/predict'
headers = {'Content-Type': 'application/json'}
payload = json.dumps(test_symptoms)

response = requests.post(url, headers=headers, data=payload)

# Tampilkan hasil
if response.status_code == 200:
    predictions = response.json()
    for i, pred in enumerate(predictions, start=1):
        print(f"[{i}] Disease: {pred['disease']}")
        print(f"    Probability: {pred['probability']:.4f}")
        print(f"    Description: {pred['description']}")
        # print(f"    Precautions: {', '.join(pred['precautions'])}")
        print("-" * 50)
else:
    print("Error:", response.status_code)
    print(response.text)


[1] Disease: (vertigo) Paroymsal  Positional Vertigo
    Probability: 0.2800
    Description: Benign paroxysmal positional vertigo (BPPV) is one of the most common causes of vertigo — the sudden sensation that you're spinning or that the inside of your head is spinning. Benign paroxysmal positional vertigo causes brief episodes of mild to intense dizziness.
--------------------------------------------------
[2] Disease: Gastroenteritis
    Probability: 0.2600
    Description: Gastroenteritis is an inflammation of the digestive tract, particularly the stomach, and large and small intestines. Viral and bacterial gastroenteritis are intestinal infections associated with symptoms of diarrhea , abdominal cramps, nausea , and vomiting .
--------------------------------------------------
[3] Disease: Paralysis (brain hemorrhage)
    Probability: 0.1000
    Description: Intracerebral hemorrhage (ICH) is when blood suddenly bursts into brain tissue, causing damage to your brain. Symptoms usuall